# Feature Engineering

In [18]:
# Importar las librerías necesarias para el análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import warnings
from sklearn.preprocessing import MinMaxScaler

# Ignorar advertencias (warnings) para mantener una salida limpia
warnings.filterwarnings("ignore")

# Cargar el dataframe limpio.
df_cleaned = pd.read_csv('datos/df_cleaned.csv')

# Haremos una copia para mantener el original intacto.
df_featured = df_cleaned.copy()

df_featured.head()

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,DLAGU6516,POZO R013 CAÑADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,2020,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,2020,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI


### Sección 1: Creación de un Índice de Salinidad (`salinity_index`)

#### Justificación
El Análisis Exploratorio de Datos (EDA) reveló una alta correlación entre las variables que miden la mineralización del agua: `CONDUCT_mS/cm` (Conductividad), `SDT_M_mg/L` (Sólidos Disueltos Totales) y `DUR_mg/L` (Dureza). Para reducir la multicolinealidad y consolidar esta información en una única y robusta característica, crearemos un `salinity_index`.

#### Metodología
1.  **Seleccionar Variables:** Se aislarán las tres columnas mencionadas.
2.  **Escalar:** Se aplicará `MinMaxScaler` para normalizar cada variable a un rango de 0 a 1. Esto es crucial para que ninguna variable domine a las otras debido a su escala original.
3.  **Combinar:** El índice se calculará como el promedio de los tres valores escalados para cada muestra. El resultado será una nueva columna, `salinity_index`, que representa el nivel general de salinidad/mineralización.

In [19]:
# 1. Seleccionar las columnas relacionadas con la salinidad
salinity_features = ['CONDUCT_mS/cm', 'SDT_M_mg/L', 'DUR_mg/L']

# 2. Inicializar el escalador y transformar los datos
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df_featured[salinity_features])

# Convertir el resultado a un DataFrame para facilitar el cálculo
df_scaled = pd.DataFrame(scaled_features, columns=salinity_features, index=df_featured.index)

# 3. Calcular el promedio de las características escaladas y asignarlo a la nueva columna
df_featured['salinity_index'] = df_scaled.mean(axis=1)

print("Se ha creado la característica 'salinity_index'.")
df_featured[['CONDUCT_mS/cm', 'SDT_M_mg/L', 'DUR_mg/L', 'salinity_index']].head()

Se ha creado la característica 'salinity_index'.


,CONDUCT_mS/cm,SDT_M_mg/L,DUR_mg/L,salinity_index
0,940.0,603.6,213.7320,0.035389
1,608.0,445.4,185.0514,0.026252
2,532.0,342.0,120.7190,0.018808
3,686.0,478.6,199.8790,0.029094
4,1841.0,1179.0,476.9872,0.077085


### Sección 2: One-Hot Encoding de la Variable `SEMAFORO`

#### Justificación
La columna `SEMAFORO` es una variable categórica nominal que resume la calidad general del agua. Los algoritmos de clustering como K-Means no pueden procesar datos de texto directamente. El One-Hot Encoding es la técnica adecuada para convertir estas categorías en un formato numérico que el modelo pueda utilizar, sin introducir un orden artificial entre ellas.

#### Metodología
Se utilizará la función `pd.get_dummies` de pandas para crear nuevas columnas binarias (0 o 1) para cada categoría presente en la columna `SEMAFORO`. Se añadirá un prefijo para mayor claridad.

In [20]:
# Aplicar One-Hot Encoding a la columna 'SEMAFORO'
semaforo_dummies = pd.get_dummies(df_featured['SEMAFORO'], prefix='SEMAFORO')

# Unir las nuevas columnas al DataFrame principal
df_featured = pd.concat([df_featured, semaforo_dummies], axis=1)

print("Se han creado las características One-Hot para 'SEMAFORO'.")
df_featured[['SEMAFORO', 'SEMAFORO_Amarillo', 'SEMAFORO_Rojo', 'SEMAFORO_Verde']].head()

Se han creado las características One-Hot para 'SEMAFORO'.


,SEMAFORO,SEMAFORO_Amarillo,SEMAFORO_Rojo,SEMAFORO_Verde
0,Verde,False,False,True
1,Verde,False,False,True
2,Rojo,False,True,False
3,Verde,False,False,True
4,Rojo,False,True,False


### Sección 3: Creación de un Contador de Contaminantes (`contaminant_count`)

#### Justificación
La columna `CONTAMINANTES` es un campo de texto que puede contener múltiples valores (ej. "AS, FE"). El número de contaminantes presentes en una muestra es una característica cuantitativa valiosa por sí misma, ya que indica la complejidad de la contaminación. Crear una característica numérica a partir de esta información la hace directamente utilizable por el modelo.

#### Metodología
Se aplicará una función a la columna `CONTAMINANTES`. Para cada fila:
1.  Si el valor es "Sin Contaminantes", el conteo será 0.
2.  De lo contrario, se contará el número de elementos separados por comas y se sumará 1 (ya que `n` comas separan `n+1` elementos).

In [21]:
# Definir la función para contar contaminantes
def count_contaminants(contaminant_string):
    if contaminant_string == 'Sin Contaminantes':
        return 0
    else:
        # Contar el número de comas y sumar 1 para obtener el número de contaminantes
        return contaminant_string.count(',') + 1

# Aplicar la función para crear la nueva columna
df_featured['contaminant_count'] = df_featured['CONTAMINANTES'].apply(count_contaminants)

print("Se ha creado la característica 'contaminant_count'.")
df_featured[['CONTAMINANTES', 'contaminant_count']].head(10)

Se ha creado la característica 'contaminant_count'.


,CONTAMINANTES,contaminant_count
0,Sin Contaminantes,0
1,Sin Contaminantes,0
2,"FLUO,AS,",3
3,Sin Contaminantes,0
4,"NO3,",2
5,"CF,",2
6,Sin Contaminantes,0
7,"CONDUC,NO3,",3
8,Sin Contaminantes,0
9,"DT,CF,AS,MN,FE,",6


### Sección 4: Guardar el DataFrame con las Nuevas Características

Finalmente, el DataFrame resultante, que ahora incluye las nuevas características diseñadas, se guarda en un nuevo archivo CSV para ser utilizado en la siguiente etapa del pipeline: el modelado.

In [22]:
# Guardar el dataframe con las nuevas características en un nuevo archivo
output_path = 'datos/df_featured.csv'
df_featured.to_csv(output_path, index=False)

print(f"El DataFrame con las nuevas características se ha guardado en: {output_path}")

El DataFrame con las nuevas características se ha guardado en: datos/df_featured.csv
